<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
import requests
import re
import dateutil.parser
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pickle
import sys
import datetime as dt
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
with open("movie_df.pkl", 'rb') as picklefile: 
    df = pickle.load(picklefile)

In [ ]:
def franch_flag(row):
    '''flag to indicate if movie is part of a franchise'''
    
    if len(row['Franchise(s)']) == 0:
        return 0
    else:
        return 1

In [ ]:
df['Franchise Flag'] = df.apply(franch_flag, axis = 1)

In [ ]:
def quarter(row):
    '''function to assing relase date to calendar quarter'''
    
    quarters = {'January': 'Q1','February': 'Q1','March': 'Q1','April':'Q2','May': 'Q2','June':'Q2',
               'July':'Q3','August':'Q3','September':'Q3','October':'Q4','November':'Q4','December':'Q4'}
    year = row['Release Date'].year
    month = row['Release Date'].month_name()
    return quarters[month]+str(year)

In [ ]:
df['Quarter'] = df.apply(quarter, axis = 1)

In [ ]:
#read in GDP data from csv
quarter_map_df = pd.read_csv('GDP.csv')

In [ ]:
#convert map df to dict
quarter_map = quarter_map_df.to_dict('split')

In [ ]:
#create map from data of quarter_dict
gdp_data = quarter_map['data']
gdp_data
gdp_map = {}
for x in gdp_data:
    gdp_map[x[0]] = x[1]

In [ ]:
df['GDP'] = df['Quarter'].map(gdp_map)

In [ ]:
#read in Netflix data from csv
netflix_map_df = pd.read_csv('Netflix.csv')

In [ ]:
net_map = netflix_map_df.to_dict('split')

In [ ]:
#create map from data of quarter_dict
netflix_data = net_map['data']

netflix_map = {}
for x in netflix_data:
    netflix_map[x[0]] = x[1]

In [ ]:
df['Netflix'] = df['Quarter'].map(netflix_map)

In [ ]:
df.rename(columns = {'Quarter':'QuarterYr'},inplace = True)

In [ ]:
df['Quarter'] = df['QuarterYr'].apply(lambda x: x[:2])

In [ ]:
#dropped Budget column because essentially duplicate of Budget Adj
df.drop(columns ='Budget',inplace = True)

In [ ]:
df['Days in Release Calc'] = (df['Close Date'] - df['Release Date']).dt.days + 1

In [ ]:
mask = ((df['Days in Release Calc'] != df['Days in Release']) &
        (df['Days in Release'].notnull()))

df[mask][['Title','Days in Release','Days in Release Calc','Release Date','Close Date']]

In [ ]:
# drop days in release after creating days in release - calc 
df.drop(columns = 'Days in Release',inplace = True)

In [ ]:
df[df['Budget Adj'].isnull()]

In [ ]:
#manually fill 16 missing budget values from IMDB 
#(had to pull molly's game, victoria & abdul & deathly hallows pt 2 data from other sources)
df.loc[df['Title']=='Vice','Budget Adj'] = 60000000 
df.loc[df['Title']=='Coco','Budget Adj'] = 175000000
df.loc[df['Title']=='I, Tonya','Budget Adj'] = 11000000
df.loc[df['Title']=='Hostiles','Budget Adj'] = 39000000
df.loc[df['Title']=="Molly's Game",'Budget Adj'] = 30000000
df.loc[df['Title']=='All the Money in the World','Budget Adj'] = 50000000
df.loc[df['Title']=='Victoria and Abdul','Budget Adj'] = 21000000
df.loc[df['Title']=='Leap!','Budget Adj'] = 30000000
df.loc[df['Title']=='10 Cloverfield Lane','Budget Adj'] = 15000000
df.loc[df['Title']=='The Divergent Series: Allegiant','Budget Adj'] = 110000000
df.loc[df['Title']=='Now You See Me 2','Budget Adj'] = 90000000
df.loc[df['Title']=='My Big Fat Greek Wedding 2','Budget Adj'] = 18000000
df.loc[df['Title']=='Zoolander 2','Budget Adj'] = 50000000
df.loc[df['Title']=='Gnomeo and Juliet','Budget Adj'] = 36000000
df.loc[df['Title']=='Harry Potter and the Deathly Hallows Part 1','Budget Adj'] = 125000000
df.loc[df['Title']=='Bruno','Budget Adj'] = 42000000

In [ ]:
#create Yr column from QuarterYr
df['Yr'] = df['QuarterYr'].apply(lambda x: x[2:])

In [ ]:
#create Month column
df['Month'] = df['Release Date'].dt.month_name()

In [ ]:
def month_yr(row):
    '''function to identify release month & year'''
    year = row['Release Date'].year
    month = row['Release Date'].month
    return str(month) +'-'+str(year)

In [ ]:
#create column w/month-year from release date
df['MonthYr'] = df.apply(month_yr,axis = 1)

In [ ]:
#read in monthly stock performance for Cinemark Holdings - largest public owner of theaters in US that had data
#available 2009-2018
cnk_map_df = pd.read_csv('CNK.csv')
#convert map df to dict
cnk_map_list = cnk_map_df.to_dict('split')

#creat map from data of quarter_dict
cnk_data = cnk_map_list['data']
cnk_data
cnk_map = {}
for x in cnk_data:
    cnk_map[x[0]] = x[1]

In [ ]:
df['CNK'] = df['MonthYr'].map(cnk_map)

In [ ]:
def pick_franchise(row):
    '''function to assign franchise to movie if in multiple on BoxOfficeMojo
    for example, Thor would be listed as in the Thor and MCU Franchises'''
    
    if len(row['Franchise(s)']) == 0:
        franchise = 'None'
    else:
        franchise = row['Franchise(s)'][-1]
    return franchise

In [ ]:
df['Franchise'] = df.apply(pick_franchise, axis = 1)

In [ ]:
df.loc[df['Title'] == 'Spider-Man: Homecoming','Franchise'] = 'Marvel Cinematic Universe' #correct franchise value

In [ ]:
df.loc[df['Franchise'] == 'Superman','Franchise'] = 'DC Extended Universe' #correct franchise value

In [ ]:
df.loc[df['Franchise'] == 'Thor','Franchise'] = 'Marvel Cinematic Universe' #correct franchise value

In [ ]:
franchise_counts = df['Franchise'].value_counts()

In [ ]:
franchise_counts[franchise_counts > 2]
other_franchises = list(franchise_counts[franchise_counts <= 2].index)
other_franchises.append('None') # include None

In [ ]:
df['Franchise2'] = df.loc[:,'Franchise'].replace(other_franchises,'Other')

In [ ]:
def ff2(row):
    '''Creates flag to indicate if movie is in a large franchise'''
    if row['Franchise2'] != 'Other':
        return 1
    else:
        return 0

In [ ]:
df['FF2'] = df.apply(ff2,axis = 1)

In [ ]:
#drop two null movies
df.dropna(inplace = True)

In [ ]:
# drop Franchise(s) column --> info captured in franchise flag
# drop Producer column --> exclude becuase viewing this through lense of a producer
df.drop(columns = ['Franchise(s)','Producer'],inplace = True) 

In [ ]:
actor_counts = df['Actor'].value_counts()

In [ ]:
#add actors with a movie count below threshold to other_actors list
actor_counts[actor_counts > 7]
other_actors = list(actor_counts[actor_counts <= 7].index)
other_actors.append('Unknown') # include Unknown --> actor is likely not 'important' if no page on boxofficemojo

In [ ]:
#add directors with a movie count below threshold to other_directors list
director_counts = df['Director'].value_counts()
director_counts[director_counts >= 6]
other_directors = list(director_counts[director_counts < 6].index)
other_directors.append('Unknown')

In [ ]:
#add writers with a movie count below threshold to other_writers list
writer_counts = df['Writer'].value_counts()
writer_counts[writer_counts >= 5]
other_writers = list(writer_counts[writer_counts < 5].index)
other_writers.append('Unknown')

In [ ]:
#add distributors with a movie count below threshold to other_distrib list
distrib_counts = df['Distributor'].value_counts()
other_distrib = list(distrib_counts[distrib_counts < 25].index)

In [ ]:
#add genres with a movie count below threshold to other_genre list
genre_counts = df['Genre'].value_counts()

In [ ]:
#read in genre mappings to consolidate categories  from csv
genre_map_df = pd.read_csv('Genres.csv')

In [ ]:
genre_map_dict = genre_map_df.to_dict('split')
genre_data = genre_map_dict['data']
genre_map = {}
for x in genre_data:
    genre_map[x[0]] = x[1]
# genre_map

In [ ]:
df['Genre'] = df['Genre'].map(genre_map) #decided not to create 'other genre' category

In [ ]:
#copy df before replacing wtih 'other'categories
df2 = df.copy()

#create 'other' categories for actor, director, distributor and writer columns
df2['Actor'].replace(other_actors,'Other',inplace = True)
df2['Director'].replace(other_directors,'Other',inplace = True)
df2['Writer'].replace(other_writers,'Other',inplace = True)
df2['Distributor'].replace(other_distrib,'Other',inplace = True)
df2.info()

In [ ]:
with open('modeling_data.pkl','wb') as picklefile:
    pickle.dump(df2,picklefile)